In [69]:
import os
import torch
import torch.nn as nn
import torch.utils.data as data

import importlib

from src.datasets.huge_stock_ds import spy
from src.models import basic_dnn_regression
importlib.reload(spy)
importlib.reload(basic_dnn_regression)

data_dir = "../data/kaggle-huge-stock-market-dataset"
etfs = os.path.join(data_dir, "ETFs")

ds = spy.SPYDataset(data_dir, ['Open'], 'Close')

In [70]:
train_ds, test_ds = data.random_split(ds, [int(0.8*len(ds)), len(ds)-int(0.8*len(ds))])
train_dl = data.DataLoader(train_ds, batch_size=256, shuffle=True)
test_dl = data.DataLoader(test_ds, batch_size=256, shuffle=True)

In [71]:
net = basic_dnn_regression.BasicDNNRegression()

In [72]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

for epoch in range(10):
    net.train_one_epoch(train_dl, optimizer, loss_fn, epoch)

Epoch: 1, Batch #: 10, Current Loss: 15372.296, Avg Loss: 19046.283: 100%|██████████| 10/10 [00:01<00:00,  5.04it/s]
Epoch: 2, Batch #: 10, Current Loss: 8482.211, Avg Loss: 11095.933: 100%|██████████| 10/10 [00:01<00:00,  5.04it/s]
Epoch: 3, Batch #: 10, Current Loss: 3873.439, Avg Loss: 5783.653: 100%|██████████| 10/10 [00:01<00:00,  5.02it/s]
Epoch: 4, Batch #: 10, Current Loss: 1262.683, Avg Loss: 2253.802: 100%|██████████| 10/10 [00:01<00:00,  5.06it/s]
Epoch: 5, Batch #: 10, Current Loss: 111.044, Avg Loss: 467.419: 100%|██████████| 10/10 [00:01<00:00,  5.03it/s]
Epoch: 6, Batch #: 10, Current Loss: 19.331, Avg Loss: 18.262: 100%|██████████| 10/10 [00:01<00:00,  5.05it/s]
Epoch: 7, Batch #: 10, Current Loss: 55.353, Avg Loss: 48.702: 100%|██████████| 10/10 [00:01<00:00,  5.04it/s]
Epoch: 8, Batch #: 10, Current Loss: 16.975, Avg Loss: 35.027: 100%|██████████| 10/10 [00:01<00:00,  5.03it/s]
Epoch: 9, Batch #: 10, Current Loss: 1.074, Avg Loss: 5.240: 100%|██████████| 10/10 [00:02<

In [73]:
net.test(test_dl, criterion=loss_fn)

Batch #: 3, Current Loss: 2.953, Avg Loss: 3.010: 100%|██████████| 3/3 [00:00<00:00,  5.61it/s]


3.0103448232014975